## Load Data

We have already downloaded the open source Empathetic-dialogue dataset and stored them as .npy file in ./empathetic-dialogue. By import prepare_data_seq from ./utils/data_loader.py, we can have the well-prepared data ready for model training and testing. The sample data is shown below:

In [1]:
from utils.data_loader import prepare_data_seq
data_loader_tra, data_loader_val, data_loader_tst, vocab, program_number = prepare_data_seq(batch_size=16)

                                      Opts                                      
--------------------------------------------------------------------------------
                                dataset: empathetic                             
                             hidden_dim: 100                                    
                                emb_dim: 100                                    
                             batch_size: 16                                     
                                     lr: 0.0001                                 
                          max_grad_norm: 2.0                                    
                              beam_size: 5                                      
                                  model: experts                                
                        act_loss_weight: 0.001                                  
                                    hop: 6                                      
                            

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wangshihang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Building dataset...
Saved PICKLE
LOADING empathetic_dialogue
[situation]: i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .
[emotion]: sentimental
[context]: ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .']
[target]: was this a friend you were in love with , or just a best friend ?
 
[situation]: i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .
[emotion]: sentimental
[context]: ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .', 'was this a friend you were in love with , or just a best friend ?', 'this was a best friend . i miss her .']
[ta

## Model Initialization
Next, we initialized the model by calling Transformer_experts

In [2]:
from utils import config
from model.transformer_mulexpert import Transformer_experts
from model.common_layer import make_infinite, get_input_from_batch, get_output_from_batch
import tensorflow as tf
from copy import deepcopy
from tqdm import tqdm
import os
import math
import time
import datetime
import numpy as np 

np.random.seed(0)

best_ppl = 1000
check_iter = 2000
patient = 0

data_iter = make_infinite(data_loader_tra)
save_model_path ='./saved_model/moel'

if(config.model == "trs"):
    model = Transformer(vocab,decoder_number=program_number)
elif(config.model == "experts"):
    model = Transformer_experts(vocab,decoder_number=program_number)
print("MODEL USED:",config.model)

MODEL USED: experts


## Define Training Step

As this is a multi-task learning problem, i.e., the encoders conduct (1) the emotion classification task to determine which of the 32 emotion labels the input context should lies on, and (2) obtain the context representation as the input of decoders to generate desired response (basically another classification task with vocab_size of classes). Therefore, the loss should consist of two parts, the **cross entropy loss** of both tasks. Since the target label has been given, we use "SparseCategoricalCrossentropy" method here

In [3]:
uniq_cfg_name = datetime.datetime.now().strftime("%Y")
checkpoint_prefix = os.path.join(os.getcwd(), "checkpoints")
if not os.path.exists(checkpoint_prefix):#create a path to save the model checkpoint
    print("create model dir: %s" % checkpoint_prefix)
    os.mkdir(checkpoint_prefix)

checkpoint_path = os.path.join(checkpoint_prefix, uniq_cfg_name)
if os.path.exists(checkpoint_path):#if there exist a checkpoint file, read them though
    model.load_weights(checkpoint_path)
    print("load weight from: %s" % checkpoint_path)

criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)# the criteria for loss calculation
optimizer = tf.keras.optimizers.Adam(lr=config.lr)# the method for model optimization

training_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32) #initializa method for training loss
training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')#initializa method for training accuracy
testing_loss = tf.keras.metrics.Mean('validation_loss', dtype=tf.float32)#initializa method for testing loss
testing_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('validation_accuracy')#initializa method for testing accuracy

def train_step(input_x, training = True):
    dec_batch, _, _, _, _ = get_output_from_batch(input_x)# get true label of generated tokens
    if training:
        with tf.GradientTape() as tape:
            logit, logit_prob = model(input_x, training)# model forward, and return the logits for emotion classification task and response generation task
            train_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(input_x[6], dtype=tf.int32), logit_prob)
        gradients = tape.gradient(train_loss, model.trainable_variables) # calculate gradient
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))# model optimization
        training_loss.update_state(train_loss)# update the training loss
        training_accuracy.update_state(input_x[6], logit_prob)# update the training accuracy
    else:
        logit, logit_prob = model(input_x, training)
        test_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(input_x[6], dtype=tf.int32), logit_prob)
        testing_loss.update_state(test_loss)# update the testing loss
        testing_accuracy.update_state(input_x[6], logit_prob)# update the testing accuracy

create model dir: /Users/wangshihang/Documents/Columbia/E4040_Neural_Network/e4040-2021Fall-Project-MOEL-sl4640-sw3275-tz2372/checkpoints


## Model Training

Now we can train the model by calling train_step function with the following code:

In [4]:
try:
    training_loss.reset_states()
    training_accuracy.reset_states()
    #apply tqdm method to track the training progress, the training will stop when patient>2
    for n_iter in tqdm(range(1000000)):
        train_step(next(data_iter), training = True)        
        if((n_iter+1)%check_iter==0):#print the cur loss and accuracy after <check_iter> of runs
            print("[train iter %d] [%s]: %0.3f [%s]: %0.3f  [%s]: %0.3f" %  (n_iter+1, "loss", training_loss.result(), "ppl", math.exp(training_loss.result()), "emo_acc", training_accuracy.result()))
            training_loss.reset_states()
            training_accuracy.reset_states()

            testing_loss.reset_states()
            testing_accuracy.reset_states()
            pbar = tqdm(enumerate(data_loader_val), total=179)# valid size: 5734
            #validate the model performance on validation set
            for j, test_batch in pbar:
                train_step(test_batch, training = False)
            print("[test iter %d] [%s]: %0.3f [%s]: %0.3f  [%s]: %0.3f" %  (n_iter+1, "loss", testing_loss.result(), "ppl", math.exp(testing_loss.result()), "emo_acc", testing_accuracy.result()))
            print('-' * 89)
            
            if (config.model == "experts" and n_iter<10000):
                continue
            if math.exp(testing_loss.result()) < best_ppl:# if we get new <best_ppl>, update it and save the model
                #model.save(save_model_path)
                model.save_weights(save_model_path, overwrite=True)# save model
                patient = 0 #reset the patient
                best_ppl = math.exp(testing_loss.result())
            else:# if the patient doesn't update for three runs, then early stop
                patient += 1
            if patient > 3:
                break
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

  0%|          | 1999/1000000 [1:36:34<762:01:50,  2.75s/it] 

[train iter 2000] [loss]: 6.651 [ppl]: 773.367  [emo_acc]: 0.047



  0%|          | 2000/1000000 [1:40:16<18926:38:30, 68.27s/it]

[test iter 2000] [loss]: 5.426 [ppl]: 227.278  [emo_acc]: 0.050
-----------------------------------------------------------------------------------------


  0%|          | 3999/1000000 [3:15:56<895:15:49,  3.24s/it]  

...

[train iter 22000] [loss]: 4.697 [ppl]: 109.580  [emo_acc]: 0.076



  2%|▏         | 22000/1000000 [18:37:55<20601:45:34, 75.83s/it]

[test iter 22000] [loss]: 5.060 [ppl]: 157.655  [emo_acc]: 0.055
-----------------------------------------------------------------------------------------


  2%|▏         | 22004/1000000 [18:38:14<828:21:52,  3.05s/it]  

-----------------------------------------------------------------------------------------
Exiting from training early


## Model Testing

We can also load the well-trained model and test its performance via the following code: 

In [5]:
# Initialize model
if(config.model == "trs"):
    model = Transformer(vocab,decoder_number=program_number)
elif(config.model == "experts"):
    model = Transformer_experts(vocab,decoder_number=program_number)
# load model weights from path
model.load_weights(save_model_path)
print("Testing MODEL:",config.model)

#Define the method to evaluate the model performance, i.e., accuracy & cross entropy loss
testing_loss = tf.keras.metrics.Mean('validation_loss', dtype=tf.float32)
testing_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('validation_accuracy')
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#apply tqdm method to track the testing progress
pbar = tqdm(enumerate(data_loader_tst), total=164)# test size: 5255
try:
    for j, test_batch in pbar:
        dec_batch, _, _, _, _ = get_output_from_batch(test_batch)#get true token labels of testing data
        logit, logit_prob = model(test_batch, training=False)
        test_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(test_batch[6], dtype=tf.int32), logit_prob)
        #test_pred_program = np.argmax(logit_prob.numpy(), axis=1)
        testing_loss.update_state(test_loss)
        testing_accuracy.update_state(test_batch[6], logit_prob)
    print("[test metrics] [%s]: %0.3f [%s]: %0.3f  [%s]: %0.3f" %  ("loss", testing_loss.result(), "ppl", math.exp(testing_loss.result()), "emo_acc", testing_accuracy.result()))
except:
    print('-' * 89)
    print('Exiting from training early')

Testing MODEL: experts


100%|██████████| 164/164 [03:35<00:00,  1.32s/it]

[test metrics] [loss]: 4.990 [ppl]: 146.904  [emo_acc]: 0.051


## Baseline Model

We also realized the Transformer baseline in ./model, and one can call it as baseline by changing the **config.model** argument in ./utils/config.py to "trs", and then run the same code as above